# Transform Orders Data - String to JSON
1. Pre-Process the JSON String to fix the Data Quality Issues
2. Transform JSON String to JSON Object  
3. Write Transformed Data to the Silver Schema

In [0]:
select *
from gizmobox.bronze.v_orders;

### 1. Pre-Process the JSON String to fix the Data Quality Issues

In [0]:
create or replace temporary view tv_orders_fixed
as
select value,
regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') as fixed_value
from gizmobox.bronze.v_orders;

### 2. Transform JSON String to JSON Object  
- schema_of_json function
- from_json function

In [0]:
select schema_of_json(fixed_value) as schema, fixed_value
from tv_orders_fixed limit 1;

In [0]:
select from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as json_value,
fixed_value
from tv_orders_fixed;

### 3. Write Transformed Data to the Silver Schema

In [0]:
create table if not exists gizmobox.silver.orders_json
as
select from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as json_value
from tv_orders_fixed;

In [0]:
select * from gizmobox.silver.orders_json;